In [6]:
import os
from users.base_user import UserSim
from users.llm_user import LLMUserSim
from dialogue_sim import DialogueSimulator
import pe_modules
import users
import dataloaders
import llms
import jinja2
import argparse
import yaml
import json

# Get Item Data

### Shapes:
All 8 combinations of {red,blue},{large,small},{circle,triangle}

In [7]:
with open('data/shapes.json') as f:
    shapes = json.load(f)
shapes

{'0': {'name': 'Red Large Circle', 'description': 'Red Large Circle'},
 '1': {'name': 'Red Large Triangle', 'description': 'Red Large Triangle'},
 '2': {'name': 'Red Small Circle', 'description': 'Red Small Circle'},
 '3': {'name': 'Red Small Triangle', 'description': 'Red Small Triangle'},
 '4': {'name': 'Blue Large Circle', 'description': 'Blue Large Circle'},
 '5': {'name': 'Blue Large Triangle', 'description': 'Blue Large Triangle'},
 '6': {'name': 'Blue Small Triangle', 'description': 'Blue Small Circle'},
 '7': {'name': 'Blue Small Triangle', 'description': 'Blue Small Triangle'}}

### Clothing

all 8 combinations of {Men's, Women's}, {Black, Red}, {Shoes,T-Shirt}

In [32]:
with open('data/clothing.json') as f:
    clothing = json.load(f)
clothing

{'0': {'name': "Men's Black Shoes", 'description': "Men's Black Shoes"},
 '1': {'name': "Men's Black T-Shirt", 'description': "Men's Black T-Shirt"},
 '2': {'name': "Men's Red Shoes", 'description': "Men's Red Shoes"},
 '3': {'name': "Men's Red T-Shirt", 'description': "Men's Red T-Shirt"},
 '4': {'name': "Women's Black Shoes", 'description': "Women's Black Shoes"},
 '5': {'name': "Women's Black T-Shirt",
  'description': "Women's Black T-Shirt"},
 '6': {'name': "Women's Red Shoes", 'description': "Women's Red Shoes"},
 '7': {'name': "Women's Red T-Shirt", 'description': "Women's Red T-Shirt"}}

# Get queries

In [34]:
easy_single_aspect_shape_queries = ['red','blue','large','small','circle','triangle']
easy_single_aspect_clothing_queries = ['men\'s','women\'s','red','blue','shoes','t-shirt']

# Entailement Code
TODO: Move to ItemScorer classes

#### FB mnli

In [8]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#### Sentence-Transformer Cross Encoder

In [11]:
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

In [3]:
print(torch.__version__)

1.13.0


In [4]:
import torch
torch.cuda.is_available()

False

In [5]:
!nvidia-smi

Tue Dec 12 11:53:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.06       Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   63C    P8     2W /  N/A |    228MiB /  4096MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nli_model.to(device)

premises = ['not small red circle','This item would be liked by someone who responed as follows: \n Query:  Are you looking for a small red circle? \n Response:  No']
hypotheses = ['large red circle', 'large red circle']

# Check for maximum token length
max_length = tokenizer.model_max_length

# Tokenize all pairs
inputs = tokenizer(premises, hypotheses, padding=True, truncation=True, return_tensors="pt", max_length=max_length)

# Ensure the tokenized inputs don't exceed the max length
if inputs.input_ids.size(1) > max_length:
    print(f"Warning: One or more tokenized inputs exceed the max length of {max_length} and will be truncated.")

inputs = inputs.to(device)
    
    
with torch.no_grad():
    outputs = nli_model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [17]:
predictions

tensor([[0.0306, 0.0413, 0.9282],
        [0.9491, 0.0498, 0.0011]])

In [13]:
nli_model.device

device(type='cpu')

In [203]:
inputs

{'input_ids': tensor([[    0,  3654,   650,  1275,  7922,     2,     2, 11802,  1275,  7922,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
        [    0,   713,  6880,    74,    28,  6640,    30,   951,    54, 18237,
         11469,    25,  3905,    35,  1437, 50118, 44489,    35,  1437,  3945,
            47,   546,    13,    10,   650,  1275,  7922,   116,  1437, 50118,
         19121,    35,  1437,   440,     2,     2, 11802,  1275,  7922,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [14]:
outputs

Seq2SeqSequenceClassifierOutput(loss=None, logits=tensor([[-1.2256, -0.9256,  2.1880],
        [ 3.1643,  0.2160, -3.5888]]), past_key_values=((tensor([[[[ 3.5664e-01,  3.4130e-01,  2.2136e+00,  ...,  1.1739e+00,
            3.2585e+00, -1.4553e+00],
          [ 3.8734e-01,  4.8377e-01,  2.1573e+00,  ...,  1.6858e+00,
            3.9412e+00, -1.9962e+00],
          [-1.9934e+00, -8.3462e-01, -1.7715e+00,  ..., -3.8792e+00,
           -4.7412e+00, -2.3865e+00],
          ...,
          [-7.2588e-01,  5.1913e+00, -2.5575e-02,  ..., -3.7220e-01,
           -5.8379e+00,  2.9871e+00],
          [ 3.0169e+00, -3.8476e-01, -2.0491e+00,  ...,  9.3332e-01,
           -1.9529e+00,  4.0722e+00],
          [ 2.4724e+00, -3.3403e+00, -3.0084e+00,  ...,  2.1397e+00,
           -1.3611e+00,  3.5844e+00]],

         [[-2.2513e+00, -2.5750e+00, -1.9645e-01,  ..., -6.4791e-01,
            3.8783e-02,  7.9893e-01],
          [-5.3976e-01, -1.9856e+00, -1.9493e-01,  ..., -1.2245e+00,
            3.7229e-0

In [15]:
premise = 'not small red circle'
hypothesis = 'large red circle'
x = tokenizer.encode(premise, hypothesis, return_tensors='pt')
#TODO: set truncation warnings
logits = nli_model(x)[0]

In [209]:
x

tensor([[    0,  3654,   650,  1275,  7922,     2,     2, 11802,  1275,  7922,
             2]])

In [210]:
logits

tensor([[-1.2256, -0.9256,  2.1880]], grad_fn=<AddmmBackward0>)

In [172]:
premise = 'large red circle'
hypothesis = '''
This item would be liked by someone who responed as follows:
Query:  Are you looking for a small red circle? \n Response:  No'''
x = tokenizer.encode(premise, hypothesis, return_tensors='pt')
#TODO: set truncation warnings
logits = nli_model(x)[0]

In [173]:
logits

tensor([[ 2.7382, -1.1287, -2.2011]], grad_fn=<AddmmBackward0>)

In [165]:
#Anton observation dec 12:
#logits are [contradicts, neutral, entails]

#Aspect negation format works OK
#-------------------------------
#premise = 'not small red circle'
#hypothesis = '''
#large red circle'''
#logits: [-0.8607, -0.2254,  1.5746]

#Raw query response format works bad
#-------------------------------
#premise = 'large red circle'
#hypothesis = '''
#This item would be liked by someone who responed as follows:
#Query:  Are you looking for a small red circle? \n Response:  No'''
#[ 2.7382, -1.1287, -2.2011]
#Anton observation Dec 12: neutral may only be output when the hypothesis is a generalization of the premise (e.g. P = soccer, H = world cup soccer)

In [127]:
import torch
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

In [108]:
test_CE_input =  [('something other than a small red circle', 'Red Large Circle'), 
                  ('Query:  Are you looking for a small red circle? \n Response:  No', 'Red Large Circle')]



In [109]:
output = model.predict(
            test_CE_input
        )

In [121]:
test_CE_input =  [('Query:  Do you prefer red items? \n Response:  Yes', 'Red Large Circle'), 
                  ('Query:  Do you prefer red items? \n Response:  Yes', 'Red Large Triangle')]



In [122]:
output = model.predict(
            test_CE_input
        )

In [123]:
torch.sigmoid(torch.from_numpy(output))

tensor([0.0146, 0.0141])

In [124]:
torch.sigmoid(torch.from_numpy(output))

tensor([0.0146, 0.0141])